# Introduction

Create a script for downloading INCA data.

**Website only allows for 2-3 months worth of data to be downloaded at a time, depending on the size of the lat-lon box.**

TODO:
* create option for downloading timeseries from lat lon location

Example of timeseries query:
https://forms.hub.zamg.ac.at/v1/timeseries/d512d5b5-4e9f-4954-98b9-806acbf754f6/historical?anonymous=true&parameters=GL,P0,RH2M,RR,T2M,TD2M,UU,VV&start=2011-03-17%2000:00&end=2021-12-01%2009:00&lon=12.9&lat=50&output_format=csv

# Setup

## Modules

In [1]:
import requests
import urllib
from pathlib import Path
import datetime
from dateutil.relativedelta import relativedelta
import subprocess
import pandas as pd

## Global variables

In [2]:
#ODIR = "/home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/INCA"
#ODIR = "/home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/precip/grids/INCA"
ODIR = "/home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA"
ODIR = Path(ODIR)

if not ODIR.is_dir():
    ODIR.mkdir(parents=True)
    
# variable
#params = "T2M" # air temperature 2m above ground
#params = "RR" # 1-hour precipitation sum
params = "GL" # global radiation 

overwrite=False
overwriteMerge=False
verbose=True

startYear = 2011 # inclusive
endYear = 2021 # inclusive

datetimeformat = "%Y-%m-%d %H:%M"

In [3]:
# gridbox for Ötztal area
gridboxlabel = "oetztal"
lat_min = 46.6
lat_max = 47.3
lon_min = 10.5
lon_max = 11.4

## Make query

In [4]:
query = {"params":params,
         "gridboxlabel":gridboxlabel,
         "lat_min":lat_min,
         "lat_max":lat_max,
         "lon_min":lon_min,
         "lon_max":lon_max,
         "output_format":"netcdf",
         "file_extention":"nc",
         "output_filename_head":"incal-hourly"}

## Save/Load query to/from file

In [5]:
queryTable = pd.DataFrame.from_dict(query,orient="index",columns=["dict"])
queryTable.to_csv("test.txt",sep="\t")

In [6]:
pd.read_table("test.txt",index_col=0).to_dict()["dict"]

{'params': 'GL',
 'gridboxlabel': 'oetztal',
 'lat_min': '46.6',
 'lat_max': '47.3',
 'lon_min': '10.5',
 'lon_max': '11.4',
 'output_format': 'netcdf',
 'file_extention': 'nc',
 'output_filename_head': 'incal-hourly'}

# Functions

In [7]:
def makeFilename(start,end,**query):
    """
    Make INCA filename from a ZAMG data hub query.
    """
    output_filename_head = query.get('output_filename_head',"incal-hourly")
    params = query.get("params","data")
    gridboxlabel = query.get("gridboxlabel","latlonbox")
    file_extention = query.get("file_extention","nc")
    
    # compact the datetime notation
    s = start.replace("-","").replace(" ","").replace(":","")
    e = end.replace("-","").replace(" ","").replace(":","")
    timeslice = f"{s}-{e}"
    # make filename
    filename = "_".join([output_filename_head,params,gridboxlabel,timeslice])+f".{file_extention}"
    return filename

def makeURL(start,end,**query):
    """
    Makes a URL string for requesting INCA_L dataset from ZAMG data hub (https://data.hub.zamg.ac.at).
    
    Default parameters requests 2-meter air temperature in a lat-lon box of the Ötztal Alps.
    
    Parameters
    ----------
    start : str
    end : str
    params : str
        default: "T2M"
    lat_min : str ; float
        default: 46.6
    lat_max : str ; float
        default: 47.3
    lon_min : str ; float
        default: 10.5
    lon_max : str ; float
        default: 11.4
    output_format : str
        default: "netcdf"
    
    Returns
    -------
    url : str
    """
    # unpack
    lat_min = query.get("lat_min",46.6)
    lat_max = query.get("lat_max",47.3)
    lon_min = query.get("lon_min",10.5)
    lon_max = query.get("lon_max",11.4)
    params = query.get("params","TD2M")
    output_format = query.get('output_format',"netcdf")
    
    # make start- and endtime strings
    sd = start.replace(" ","%20")
    ed = end.replace(" ","%20")

    # make gridbox string
    bbox = f"{lat_min},{lon_min},{lat_max},{lon_max}"

    url = "https://forms.hub.zamg.ac.at/v1/grid/d512d5b5-4e9f-4954-98b9-806acbf754f6/" + \
            f"historical?anonymous=true&parameters={params}&start={sd}&end={ed}&bbox={bbox}&output_format={output_format}"

    return url

def makeTimeSlices(year,firstMonth = 1,lastMonth=12,maxMonths=2,datetimeformat = "%Y-%m-%d %H:%M"):
    """
    Makes a list of time slices (start and end times) for a specified year.
    
    Parameters
    ----------
    year : int
    firstMonth : int
        default: 1
    lastMonth : int
        default: 12
    maxMonth : int
        default: 2
    datetimeformat : str
        default: '%Y-%m-%d %H:%M'
    """
    # set the last day of the year
    firstDOY = datetime.datetime(2011,3,1,0,0) # data is only available from March 2011 onwards
    lastDOY = datetime.datetime(year+1,1,1,0,0)
    # set start month
    month = firstMonth
    
    # make list of time slices
    slices = []
    while month<lastMonth+1:
        dtStart = datetime.datetime(year,month,1,0,0)
        # check the first
        if dtStart<firstDOY:
            dtStart = firstDOY
        dtEnd = dtStart + relativedelta(months=+maxMonths)
        # check that the end datetime does not exceed the last day of the year
        if dtEnd >= lastDOY:
            dtEnd = lastDOY
        
        # convert to string
        start = dtStart.strftime(datetimeformat)
        end = dtEnd.strftime(datetimeformat)

        slices.append((start,end))

        month = month + maxMonths
    
    return slices

def downloadData(start,end,ODIR,overwrite=False,verbose=True,**query):
    """
    Requests and downloads data from ZAMG data hub, and saves the file in a specifed directory.
    """
    # unpack
    lat_min = query.get("lat_min",46.6)
    lat_max = query.get("lat_max",47.3)
    lon_min = query.get("lon_min",10.5)
    lon_max = query.get("lon_max",11.4)
    
    # make filename
    filename = makeFilename(start,end,**query)
    outfile = ODIR.joinpath(filename)
    
    # check whether file already exists
    if overwrite or not outfile.is_file():
        url = makeURL(start,end,**query)
        r = requests.get(url)
        if str(r) == "<Response [400]>":
            raise requests.HTTPError(f"{r}: The data slice that you requested is too big!")
        ff,html = urllib.request.urlretrieve(url, outfile)
        if verbose: print(filename, "was downloaded.")
    else:
        if verbose: print(filename, "has already been downloaded:",outfile)
        
    return outfile

def mergeNetCDFfilesByYear(year,DIR,verbose=True,overwrite=False):
    """
    Merge NetCDF files from same year in a specified directory using cdo.
    
    Parameters
    ----------
    year : int or string
    DIR : str or pathlib.PosixPath
    verbose : boolean
        default True
    overwrite : boolean
        default False
    """
    # convert to PosixPath
    DIR = Path(DIR)
    # get files from directory
    files = [str(f) for f in list(DIR.glob(f"*_{year}*00.nc"))]
    # make outfile
    outfile = Path(files[0]).name.split(f"_{year}")[0]+f"_{year}.nc"
    # make command arguments
    if not overwrite:
        cmd = ["cdo","mergetime"] + files + [str(ODIR.joinpath(outfile))]
    else:
        cmd = ["cdo","-O","mergetime"] + files + [str(ODIR.joinpath(outfile))]
    # make string of argument list
    cmd = " ".join(cmd)
    # run process
    process = subprocess.run(cmd,shell=True,capture_output=True,universal_newlines=True)
    # print output
    if verbose:
        print(">>>",cmd)
        print(process.stderr)

# Download data

In [8]:
# get all in a year
for year in range(startYear,endYear+1):
    slices = makeTimeSlices(year)
    for start,end in slices:
        downloadData(start,end,ODIR,overwrite=overwrite,verbose=verbose,**query)

incal-hourly_GL_oetztal_201103010000-201105010000.nc was downloaded.
incal-hourly_GL_oetztal_201103010000-201105010000.nc has already been downloaded: /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201103010000-201105010000.nc
incal-hourly_GL_oetztal_201105010000-201107010000.nc was downloaded.
incal-hourly_GL_oetztal_201107010000-201109010000.nc was downloaded.
incal-hourly_GL_oetztal_201109010000-201111010000.nc was downloaded.
incal-hourly_GL_oetztal_201111010000-201201010000.nc was downloaded.
incal-hourly_GL_oetztal_201201010000-201203010000.nc was downloaded.
incal-hourly_GL_oetztal_201203010000-201205010000.nc was downloaded.
incal-hourly_GL_oetztal_201205010000-201207010000.nc was downloaded.
incal-hourly_GL_oetztal_201207010000-201209010000.nc was downloaded.
incal-hourly_GL_oetztal_201209010000-201211010000.nc was downloaded.
incal-hourly_GL_oetztal_201211010000-201301010000.nc was downloaded.
incal-hourly_GL_oetzta

# Merge NetCDF files by year

In [9]:
for year in range(startYear,endYear+1):
    mergeNetCDFfilesByYear(year,ODIR,overwrite=overwriteMerge,verbose=verbose)

>>> cdo mergetime /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201105010000-201107010000.nc /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201107010000-201109010000.nc /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201111010000-201201010000.nc /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201103010000-201105010000.nc /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_201109010000-201111010000.nc /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/radiation/grids/INCA/incal-hourly_GL_oetztal_2011.nc
Warning (cdfInqContents): Coordinates variable lambert_conformal_conic can't be assigned!
Warning (cdfInqContents): Coordinates variable lambert_conformal_conic can't be assigned!
W